In [1]:
# 1. 데이터 읽기
import pandas as pd

geoje_poi = pd.read_csv('geoje_poi1.csv', index_col=False)

geoje_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1099 non-null   object 
 1   address_name         1099 non-null   object 
 2   category_group_code  1099 non-null   object 
 3   category_group_name  1099 non-null   object 
 4   category_name        1099 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1099 non-null   int64  
 7   phone                383 non-null    object 
 8   place_name           1099 non-null   object 
 9   place_url            1099 non-null   object 
 10  road_address_name    659 non-null    object 
 11  x                    1099 non-null   float64
 12  y                    1099 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 111.7+ KB


In [2]:
keywords = set()
for keyword in geoje_poi['keyword'].unique():
    for k in keyword.split(','):
        keywords.add(k)
keywords = list(keywords)

keywords

['맛집', '여행', '가볼만한 곳', '체험', '감성']

In [3]:
groups = []

for category in geoje_poi['category_group_name'].unique():
    groups.append(category)

groups

['관광명소', '카페', '문화시설']

In [5]:
geoje_poi[['id', 'place_name', 'keyword', 'category_group_name']].head(1)

,id,place_name,keyword,category_group_name
0,11026203,구조라해수욕장,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",관광명소


In [6]:
from tqdm.notebook import tqdm
geoje_poi['content'] = ''

for idx, row in tqdm(geoje_poi.iterrows()):
    content = (row['keyword']+','+row['category_group_name']).replace(' ', '').replace(',', ' ')
    geoje_poi.loc[idx, 'content'] = content

0it [00:00, ?it/s]

In [7]:
geoje_poi[['id', 'place_name', 'keyword', 'category_group_name', 'content']].head(2)

,id,place_name,keyword,category_group_name,content
0,11026203,구조라해수욕장,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",관광명소,여행 가볼만한곳 여행 가볼만한곳 여행 가볼만한곳 관광명소
1,8021411,외도보타니아,"여행,가볼만한 곳",관광명소,여행 가볼만한곳 관광명소


In [8]:
geoje_poi.loc[0, 'content']

'여행 가볼만한곳 여행 가볼만한곳 여행 가볼만한곳 관광명소'

In [10]:
geoje_poi.to_csv('geoje_poi_content.csv', index=False)

In [11]:
geoje_poi = pd.read_csv('geoje_poi_content.csv', index_col=False)
geoje_poi.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,content
0,"여행,가볼만한 곳,여행,가볼만한 곳,여행,가볼만한 곳",경남 거제시 일운면 구조라리 423-1,AT4,관광명소,"여행 > 관광,명소 > 해수욕장,해변",NaN,11026203,055-639-3000,구조라해수욕장,http://place.map.kakao.com/11026203,경남 거제시 일운면 거제대로 2056,128.69289,34.806965,여행 가볼만한곳 여행 가볼만한곳 여행 가볼만한곳 관광명소


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
content_mat = count_vect.fit_transform(geoje_poi['content'])
content_mat.shape

(1099, 23)

In [13]:
count_vect.get_feature_names()

['가볼만한곳',
 '가볼만한곳 관광명소',
 '가볼만한곳 여행',
 '감성',
 '감성 감성',
 '감성 맛집',
 '감성 카페',
 '관광명소',
 '맛집',
 '맛집 감성',
 '맛집 맛집',
 '맛집 체험',
 '맛집 카페',
 '문화시설',
 '여행',
 '여행 가볼만한곳',
 '여행 체험',
 '체험',
 '체험 가볼만한곳',
 '체험 맛집',
 '체험 문화시설',
 '체험 카페',
 '카페']

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

content_sim = cosine_similarity(content_mat, content_mat)
print(content_sim.shape)
print(content_sim[:2])

(1099, 1099)
[[1.         0.85634884 0.98882646 ... 0.         0.         0.        ]
 [0.85634884 1.         0.92376043 ... 0.         0.         0.        ]]


In [15]:
ex_poi = geoje_poi.copy(deep = True)
ex_poi['nl_cosine'] = content_sim[0]

In [16]:
ex_poi[['id', 'place_name', 'nl_cosine']].head(5)

,id,place_name,nl_cosine
0,11026203,구조라해수욕장,1.000000
1,8021411,외도보타니아,0.856349
2,7862496,망치몽돌해수욕장,0.988826
3,11297994,와현모래숲해수욕장,1.000000
4,8559153,지심도,1.000000


In [17]:
import numpy as np

np.save('nl_content_sim.npy', content_sim)

In [18]:
import numpy as np

np_file = np.load(file='nl_content_sim.npy')
np_file[0][:5]

array([1.        , 0.85634884, 0.98882646, 1.        , 1.        ])